In [1]:
#!pip install gensim

  Using cached Cython-0.29.23-cp38-cp38-win_amd64.whl (1.7 MB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


# Gensim word2vec workplow:
ref: https://youtu.be/Q2NtCcqmIww
1) prepare data
2) preprocessing data: tokenization, stop word removal... can use: simple_preprocesser from gensim.utils
3) create the word2vec model and ready to train with your own dataset
4) build_vocabulary 

In [2]:
import gensim
from gensim.models import Word2Vec
##############################################
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from gensim.utils import simple_preprocess #for preprocessing
#from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
#import seaborn as sns

In [5]:
# Prepare customized dataset
import pandas as pd
mydata = pd.read_csv('clean_train.csv')
mydata.head()
# we only want relevant columns
clean_data = pd.concat([mydata['relevant'], mydata['clean']], axis=1, keys=['relevant','clean'])
#clean_data.to_csv('clean_test_df.csv')

,relevant,clean
0,1,rt granetman hacker held open mongodb database...
1,1,vuln google chrome multiple security vulnerabi...
2,-1,thehackersnews nmerrigan 0xdude number of mong...
3,1,several thousand linux devices infected with t...
4,1,adobe flash player flaws remain the most used ...
...,...,...
4605,1,threatmeter suse sles12 security update kernel...
4606,1,vuln apache http server cve-2016-2161 denial o...
4607,-1,oracle drops plans for solaris 12
4608,1,redhat cloudforms management engine 4.1 execut...


In [6]:
# data preprocessing 
tokenzed_text = clean_data.clean.apply(simple_preprocess)
tokenzed_text # a new panda series with list of tokens 

0       [rt, granetman, hacker, held, open, mongodb, d...
1       [vuln, google, chrome, multiple, security, vul...
2       [thehackersnews, nmerrigan, xdude, number, of,...
3       [several, thousand, linux, devices, infected, ...
4       [adobe, flash, player, flaws, remain, the, mos...
                              ...                        
4605    [threatmeter, suse, sles, security, update, ke...
4606    [vuln, apache, http, server, cve, denial, of, ...
4607                 [oracle, drops, plans, for, solaris]
4608    [redhat, cloudforms, management, engine, execu...
4609    [oracle, peoplesoft, hcm, cross, site, scripting]
Name: clean, Length: 4610, dtype: object

In [ ]:
# create the w2v model with parameters 
customized_model = gensim.models.Word2Vec(
    window = 10, #similar to cnn slider  size is flexible, you can play around with it
    min_count = 2, # minimax words in a sentence
    workers = 2 #how many cpu cores you want to use to train the model
)
customized_model.build_vocab(tokenzed_text, progress_per = 1000) #show progress every 1000 steps

In [11]:
# check some parameters before training
customized_model.epochs # default is 5 
customized_model.corpus_count  #we have 4610 here

4610

In [26]:
#train the w2v model and save locally, next time you can only load the model
customized_model.train(tokenzed_text, total_examples = customized_model.corpus_count, epochs = customized_model.epochs)
#customized_model.save("customized_model.model")

(108561, 145998)

In [23]:
# explore the model by using model.wv.most_similar()
print(customized_model.wv.most_similar('oracle'))
print(customized_model.wv.most_similar('ransomware'))
print(customized_model.wv.similarity(w1 = "mysql", w2 = "oracle"))
print(customized_model.wv.similarity(w1 = "cloud", w2 = "server"))

[('se', 0.9583156704902649), ('fusion', 0.9280099868774414), ('middleware', 0.9256894588470459), ('jre', 0.9007167220115662), ('java', 0.8971377015113831), ('business', 0.8917595148086548), ('jrockit', 0.8887305855751038), ('mysql', 0.869497537612915), ('telephony', 0.8682535290718079), ('production', 0.8675521612167358)]
[('down', 0.9944398999214172), ('learning', 0.9943986535072327), ('hacked', 0.9942374229431152), ('align', 0.993852436542511), ('urge', 0.9937928915023804), ('https', 0.9936993718147278), ('researchers', 0.9936281442642212), ('turkey', 0.993478000164032), ('hacks', 0.9933807849884033), ('claims', 0.9933537244796753)]
0.86949754
0.56539226
